In [ ]:
#Comparing Search Algorithms - CMA-ES
#Disease Biophysics Group
#Written by John Zimmerman
#Updated 4/22/21

import os
import tensorflow as tf
import numpy as np
from itertools import product
import pandas as pd
#from scipy import interpolate
import scipy.interpolate
import scipy.stats
import math
from matplotlib.pyplot import *
%matplotlib inline
import h5py
import GeoSwimmer
import statsmodels.api as sm
from cmaes import CMA, get_warm_start_mgd


print(tf.__version__)
tf.compat.v2.keras.backend.clear_session()

In [ ]:
def GenFooDataframe(dnaLength):
    #---Make Swimmer Dataframe with Foo Velocity Labels ---
    #Takes ~15 min to run
    print('Generating Foo dataframe. Takes ~15 min')
    #Generate List of all possible swimmers
    dnaAlphabet = SwimDNA.DNABasis()
    fullswimlist = [''.join(i) for i in product(dnaAlphabet, repeat = dnaLength)]
    fullswimlist = fullswimlist[1:-1]
    df = pd.DataFrame({'DNA':fullswimlist})

    #SampleSwim = Swimlist.sample(100000,random_state=seed)
    print('Label Velocities...')
    dfLabels = df.DNA.apply(SwimVel.gen_approx_swim_vels, args=(.5,0.0,0.0,0,.05)) 
    print('Done')

    alphabetVectors = RadarPlot.constructRadarBasis(SwimDNA.DNABasis())
    print('Radar Points...')
    RadarPoints = df.DNA.apply(RadarPlot.DNARadarPointsList, args=(alphabetVectors,))
    RadarPoints = pd.DataFrame(RadarPoints.to_list(),columns=['RadX','RadY'])
    print('Done')

    df['RadX']=RadarPoints.RadX.to_list()
    df['RadY']=RadarPoints.RadY.to_list()
    df['Label'] = dfLabels
    return df

class SwimSearch:
    def seededList(df):
        alphabet = SwimDNA.DNABasis()
        seedlist=  np.array([])
        for a in alphabet[1:-1]:
            seedlist = np.append(seedlist,(a*6))
            seedlist = np.append(seedlist,a+alphabet[0]*5)
            seedlist = np.append(seedlist,a+alphabet[-1]*5)
        
        seed = pd.DataFrame({'DNA':seedlist})
        seed = df[df.DNA.isin(seed.DNA)]
        
        return seed

In [ ]:
class SwimDNA:
    def getWeights(DNA):
        weights = np.ones(len(DNA))
        m = -1/float(len(DNA))
        b = 1
        for i in range(0,len(DNA)):
            weights[i] = m*i+b
        weights = weights/np.sum(weights)
        return weights

    def string_to_matrix(string):
        matrix = np.zeros((14,len(string)), dtype=np.int8)
        for position, letter in enumerate(string):
            num = (ord(letter.upper()) - 65) % 14
            matrix[num, position] = 1
        return matrix

    
    def DNABasis():
        return ['A','B','C','D','E','F','G','H','I','J','K','L','M','N']
    
    def avgDnaListDistance(dnalist):
        #Feed in dataframe.DNA list
        avglist = np.zeros(dnalist.size) #Declare List for storing Averages

        for DNA1,i in zip(dnalist,np.arange(dnalist.size)): #Cycled through each DNA
            dist = 0
            for DNA2 in dnalist: #Compare to Each Other DNA in database
                if DNA1 != DNA2:
                    dist += SwimDNA.dnaDistance(DNA1,DNA2)
            dist = dist/float(dnalist.size-1) #Average Out Values
            avglist[i] = dist
        return avglist

    def dnaDistance(DNA1,DNA2):
        vector1 = SwimDNA.DNAVector(DNA1)
        vector2 = SwimDNA.DNAVector(DNA2)
        dist = 0
        for a in SwimDNA.DNABasis():
            dist += (vector1[a]-vector2[a])**2

        return np.sqrt(dist)

    def DNAVector(DNA):
        weights = SwimDNA.getWeights(DNA)
        vector = {a:0 for a in SwimDNA.DNABasis()}
        num = np.arange(len(DNA))

        for a,i in zip(DNA,num):
                    vector[a]+=1*weights[i]
        return vector


In [ ]:
class NN:
    
    def OneHotEncodeList (SwimList):
        trainMatrix  =np.zeros((SwimList.DNA.size,14,len(SwimList.DNA.iloc[0])))

        i=0
        for DNA in SwimList.DNA:
            trainMatrix[i,:,:] = SwimDNA.string_to_matrix(DNA)
            i+=1

        trainMatrix= tf.convert_to_tensor(trainMatrix,dtype=tf.float16)    
        return trainMatrix

    def LabelTensor (SwimList):
        labelMatrix  =np.zeros(SwimList.Label.size)

        i=0
        for DNA in SwimList.DNA:
            trainMatrix[i,:,:] = SwimDNA.string_to_matrix(DNA)
            i+=1

        trainMatrix= tf.convert_to_tensor(trainMatrix,dtype=tf.float32)    
        return trainMatrix

    def genNNModel(InputLength):
        model = tf.keras.models.Sequential([
          tf.keras.layers.Flatten(input_shape=(14, InputLength)),
          tf.keras.layers.Dense(14, activation='relu'),
          tf.keras.layers.Dropout(0.2),
          tf.keras.layers.Dense(14*14, activation='relu'),
          tf.keras.layers.Dropout(0.2),
          tf.keras.layers.Dense(14, activation='relu'),
          tf.keras.layers.Dense(1,activation='sigmoid')
        ])
        return model
   
    def genNN_MatMul_model(InputLength):# define two sets of inputs
        inputA = tf.keras.Input(shape=(14,InputLength))

        # Dense Input Branch - Make a 14 dim vector
        x = tf.keras.layers.Flatten()(inputA)
        x = tf.keras.layers.Dense(14, activation="relu")(x)
        x = tf.keras.layers.Reshape((14, 1))(x)
        #x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.Model(inputs=inputA, outputs=x)
        
        #Build cross-correlation Matrix
        y = tf.keras.layers.Flatten()(inputA)
        y = tf.keras.layers.Dense(14*14, activation="relu")(y)
        y = tf.keras.layers.Reshape((14, 14))(y)
        #x = tf.keras.layers.BatchNormalization()(x)
        y = tf.keras.Model(inputs=inputA, outputs=y)

        
        #combine the output of the two branches
        combined = tf.linalg.matmul(y.output, x.output)

        # Combined Outputs
        z = tf.keras.layers.Flatten()(combined)
        z = tf.keras.layers.Dense(14, activation="relu")(z)
        z = tf.keras.layers.Dropout(0.2)(z)
        z = tf.keras.layers.Dense(6, activation="relu")(z)
        z = tf.keras.layers.Dense(1, activation="sigmoid")(z)


        #Combined model outputs join of branches
        model = tf.keras.models.Model(inputs=[inputA], outputs=z)
        return model


    def genNN_mixed_model(InputLength):# define two sets of inputs
        inputA = tf.keras.Input(shape=(14,InputLength))

        # Dense Input Branch
        x = tf.keras.layers.Flatten()(inputA)
        x = tf.keras.layers.Dense(14, activation="relu")(x)
        #x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.Model(inputs=inputA, outputs=x)
        
        #inputB = tf.reshape(inputA,(-1,14,InputLength,1))
        # the second branch opreates on the second input
        y = tf.keras.layers.Reshape((-1, -1, 1))(inputA)
        y = tf.keras.layers.Conv2D(16, (5, 5), strides=(2, 2), padding='same')(y)
        y = tf.keras.layers.Conv2D(32, (5, 5), strides=(2, 2), padding='same')(y)
        #y = tf.keras.layers.BatchNormalization()(y)
        y = tf.keras.layers.Dropout(0.2)(y)
        y = tf.keras.layers.Conv2DTranspose(32, (5, 5), strides=(2, 2), padding='same', use_bias=False)(y)
        y = tf.keras.layers.Conv2DTranspose(16, (5, 5), strides=(2, 2), padding='same', use_bias=False)(y)
        #y = tf.keras.layers.BatchNormalization()(y)
        y = tf.keras.layers.Dropout(0.2)(y)
        y = tf.keras.layers.Reshape((14, InputLength))(inputA)
        y = tf.keras.layers.Flatten()(y)
        y = tf.keras.layers.Dense(14, activation="relu")(y)
        #y = tf.keras.layers.BatchNormalization()(y)
        y = tf.keras.Model(inputs=inputA, outputs=y)
        
        #combine the output of the two branches
        combined = tf.keras.layers.add([x.output, y.output])
        #combined = tf.keras.layers.concatenate([x.output, y.output])

        # Combined Outputs
        z = tf.keras.layers.Flatten()(combined)
        z = tf.keras.layers.Dense(14, activation="relu")(z)
        z = tf.keras.layers.Dropout(0.2)(z)
        z = tf.keras.layers.Dense(6, activation="relu")(z)
        z = tf.keras.layers.Dense(1, activation="sigmoid")(z)


        #Combined model outputs join of branches
        model = tf.keras.models.Model(inputs=[inputA], outputs=z)
        return model
    
    def genNNModelLim(InputLength):
        tf.keras.backend.set_floatx('float64')
        model = tf.keras.models.Sequential([
          tf.keras.layers.Flatten(input_shape=(14, InputLength)),
          tf.keras.layers.Dense(14, activation='relu'),
          tf.keras.layers.Dropout(0.1),
          #tf.keras.layers.Dense(32, activation='relu'),
          #tf.keras.layers.Dropout(0.1),
          tf.keras.layers.Dense(14, activation='relu'),
          tf.keras.layers.Dropout(0.1),
          #tf.keras.layers.Dense(64, activation='relu'),
          tf.keras.layers.Dense(14, activation='relu'),
          tf.keras.layers.Dense(1,activation='sigmoid')
        ])
        return model
    
    def NNOptimizer(lr=0.001,epsilon=1e-07,amsgrad=False):
        return tf.keras.optimizers.Adam(learning_rate=lr, epsilon=epsilon, amsgrad=amsgrad, name='Adam')
    
    def cum_KL_divergence(ge_c,ge_mean,ge_sigma):
        def KL_div(cdf_pred, cdf_true):
            P = NN.TF_pdf(NN.TF_ppf(cdf_pred,ge_c,ge_mean,ge_sigma),ge_c,ge_mean,ge_sigma) #CDF -> PDF
            Q = NN.TF_pdf(NN.TF_ppf(cdf_true,ge_c,ge_mean,ge_sigma),ge_c,ge_mean,ge_sigma)  #CDF -> PDF
            P = tf.where(tf.math.is_nan(P),tf.convert_to_tensor(0.000001,dtype=tf.float64),P)
            Q = tf.where(tf.math.is_nan(Q),tf.convert_to_tensor(0.000001,dtype=tf.float64),Q)

            return tf.math.square(cdf_pred-cdf_true)#+P*tf.math.log(P/Q)
        return KL_div

    
    def TF_ppf(q,c,mean,sigma):
        x=-tf.math.log(-tf.math.log(q))
        
        return tf.where((c == 0) & (x == x),x,-tf.math.expm1(-c *x )/c)*sigma+mean
    
    def TF_pdf(x,c,mean,sigma):
        z = (x-mean)/sigma
        return tf.where((c==0),tf.math.exp(z)*tf.math.exp(-1*tf.math.exp(z)),tf.math.exp(-(1-c*z)**(1/c))*(1-c*z)**(1/c-1))*(1/sigma)

In [ ]:
def Evaluate(truth,label):
    return np.sqrt((truth-label)**2).sum()

def Train_Loop(savepath,loadpath, seed=12345, InitialNumbers=1000,testNum=1000,epochsperloop=100,loops=10,simsperloop=100, save=False,method='Random',lr=0.001,epsilon=1e-07):

        df = pd.read_pickle(loadpath+'results2.pkl')
        dnaLength = len(df.DNA[0])
        #print(f'DNAL: {dnaLength}')

        #Initial Training Data/ Test Data Sampling
        train = SwimSearch.seededList(df) #Seed with some selected basis functions (BAAAAA,BNNNNN,BAAAAA)
        remain = InitialNumbers-train.DNA.size
        train = train.append(df[~df.DNA.isin(train.DNA)].sample(remain,random_state=seed)) 
        
        #Initialize Model
        #optimizer = CMA(mean=)
        
        if save:
            TopPercentile = np.zeros(loops)

        for loopnum in range(0,loops):
            print(f'Train min Value: {train.Label.min()}')
            print(f'Train Max Value: {train.Label.max()}')

            #Reset Model - Reloading Weights
            model.load_weights(savepath+'model.h5')

            #Prepare Samples for Model
            test = df[~df.DNA.isin(train.DNA)].sample(testNum,random_state=seed+1+loopnum*simsperloop) #Generate Test Set of Swimmers
            rescale_max =  train.Label.max()*1.5
            rescale_min =  train.Label.min()*1.5

            OH_train = NN.OneHotEncodeList(train) #One Hot encoded tensor
            OH_test = NN.OneHotEncodeList(test) #One Hot encoded tensor
            #train_label = tf.convert_to_tensor(train.Label.to_numpy(),dtype=tf.float32)
            #test_label = tf.convert_to_tensor(test.Label.to_numpy(),dtype=tf.float32)
            train_label = tf.convert_to_tensor(SwimSearch.normalizeArray(train.Label.to_numpy(),rescale_min,rescale_max),dtype=tf.float32)
            test_label = tf.convert_to_tensor(SwimSearch.normalizeArray(test.Label.to_numpy(),rescale_min,rescale_max),dtype=tf.float32)

            #Fit Model to data
            model.fit(OH_train, train_label, epochs=epochsperloop)
            print(f'Loop {loopnum} MSE: {np.average((test.Label.to_numpy()-SwimSearch.rescaleArray(model(OH_test,training=False).numpy(),rescale_min,rescale_max))**2)}')

            #Update Model Label
            print('Updating Model Label...')
            df['ModelLabel'] = SwimSearch.rescaleArray(model(OH_df,training=False).numpy(),rescale_min,rescale_max)
            #df['ModelLabel'] = model(NN.OneHotEncodeList(df)).numpy()
            
            
            
            train = searchmethod(df,train,simsperloop,loopnum,seed)
            
            if save:
                sample = df.sample(50000,random_state=seed)
                OH_sample = NN.OneHotEncodeList(sample)
                fig = RadarPlot.PlotRadarMesh(sample.RadX,sample.RadY,sample.ModelLabel,figwidth=9,figheight=6.5,vmin=df.Label.min(),vmax=df.Label.max(),res=100)
                savefig(savepath+'Model_'+method+'_seed'+str(seed)+'_LoopNum'+str(loopnum)+'.png')
                clf()
                close()
                
                TopPercentile[loopnum] = SwimSearch.top_predicted(df,2000)
            
        
        if save:
            np.savetxt(savepath+'TopPerc2000_'+method+'_seed'+str(seed)+'.txt',TopPercentile)
        #return train
        return model, df, rescale_min, rescale_max

In [ ]:
def InitializeTraining(df,initialNumber,seed):
    train = SwimSearch.seededList(df) #Seed with some selected basis functions (BAAAAA,BNNNNN,BAAAAA)
    remain = initialNumber-train.DNA.size
    train = train.append(df[~df.DNA.isin(train.DNA)].sample(remain,random_state=seed))
    return train

In [ ]:
train = InitializeTraining(df,InitialtrainNum,seedlist[0])

In [ ]:
def Evaluate_NumSeq(num_sequence, target, df):
    #print(num_sequence)
    DNA = SequenceToDNA(num_sequence)
    print(DNA)
    label = df[df.DNA==DNA].Label.to_list()[0]
    return np.sqrt((target-label)**2)

def Evaluate_DNA(DNA, target, df):
    print(DNA)
    label = df[df.DNA==DNA].Label.to_list()[0]
    return np.sqrt((target-label)**2)

def DNA_ToArray(DNA):
    seq = np.zeros(len(DNA))
    for (letter,i) in zip(DNA,range(0,len(DNA))):
        seq[i] = ord(letter)-65
    return seq
        
def SequenceToDNA(seq):
    int_seq = np.rint(seq)
    string = ''
    for i in int_seq:
        string += chr(65+int(i))
    return string

def Eval_top_predicted(evaluated_DF, dataframe,num=2000):
    topLabel = dataframe.nlargest(num,"Label")
    #topModel = dataframe.nlargest(num,"ModelLabel")
    correct = evaluated_DF[evaluated_DF.DNA.isin(topLabel.DNA)==True].DNA.size
    print(f'correct: {correct}')
    return correct

#DNA_ToArray('NGGGGG')

In [ ]:
def CMAES_TrainLoop(savepath,loadpath, seed=12345, sigma = 1.5, InitialNumbers=400,loops=10,simsperloop=50, save=False):
    initial_mean = np.zeros(6)
    initial_mean[:] = 7
    target = df.nlargest(1,"Label").Label.to_list()[0]
    bounds = np.array([[0,13],[0,13],[0,13],[0,13],[0,13],[0,13]])
    optimizer = CMA(mean=initial_mean, sigma=sigma, bounds=bounds, seed=seed)
    
    initial_list = InitializeTraining(df,InitialNumbers,seed)
    #initial_list = InitializeTraining(df,41,10)
    evaluated_list = initial_list.copy()
    evaluated_list['Gen'] = -1

    
    
    if save:
        print('TestSave')
        evaluated_list.to_pickle(savepath+'CMAES_Results_'+str(seed)+'.pkl',)
    
    solutions = []
    for DNA in evaluated_list.DNA.to_list():
        seq = DNA_ToArray(DNA)
        val = Evaluate_DNA(DNA,target,df)
        solutions.append((seq, val))
    #optimizer.tell(solutions)
    
    #Warm Start Optimizer
    ws_mean, ws_sigma, ws_cov = get_warm_start_mgd(solutions) 
    optimizer = CMA(mean=ws_mean, sigma=sigma, cov=ws_cov,bounds=bounds, seed=seed, population_size=simsperloop)
    
    Eval_top_predicted(evaluated_list,df,2000)
    evaluated_list['Gen'] = -1
    
    Predicted = np.array([])
    
    for generation in range(loops):
        print(f'Generation: {generation}')
        solutions = []
        for _ in range(optimizer.population_size):
            x = optimizer.ask()
            print(x)
            DNA = SequenceToDNA(x)
            value = Evaluate_DNA(DNA,target,df)
            solutions.append((x, value))
            
            append_copy = df[df.DNA == DNA].copy()
            append_copy['Gen'] = generation
            evaluated_list = evaluated_list.append(append_copy)
            

        optimizer.tell(solutions)
        correct = Eval_top_predicted(evaluated_list,df,2000)
        Predicted = np.append(Predicted,correct)
        
        if save:
            print('Saving')
            evaluated_list.to_pickle(savepath+'CMAES_Results_'+str(seed)+'.pkl')
            np.savetxt(savepath+'CMAES_predicted_'+str(seed)+'.txt',Predicted)

In [ ]:
#Run An Example Training Model

InitialtrainNum = 100#400
loops = 8+1+6 #Plus 1 to hit full Amount of 800 Swimmers
simsperloop = 50#50
save = True
loadpath = 'G:\\...\\'
savepath = 'G:\\...\\'
sigma = 13/5

seedlist = [1009,1013,1019,1021,1031,1033,1039,1049,1051,1061] #First 10 prime numbers with 4 digits


In [ ]:
loadpath = 'G:\\...\\'
#e.g. 'G:\\Data Process\\NeuralNet\\EvaluationMethods\\'

loadcheck = os.path.exists(loadpath+'results.pkl')
if loadcheck == True:
    df = pd.read_pickle(loadpath+'results.pkl')
    print('Results loaded')
else:
    df = GenFooDataframe(6)
    df.to_pickle(loadpath+'results.pkl')

dnaLength = len(df.DNA[0])

In [ ]:
loadpath = 'G:\\...\\'
savepath = 'G:\\...\\'
#e.g. 'G:\\Data Process\\NeuralNet\\EvaluationMethods\\'

for seednum in seedlist:
    CMAES_TrainLoop(savepath,loadpath, seed=seednum, sigma = sigma, InitialNumbers=InitialtrainNum,loops=loops,simsperloop=simsperloop, save=save)

In [ ]:
DF_name = 'CMAES_Results_1061'
cmaes = pd.read_pickle(savepath+DF_name+'.pkl')

In [ ]:
m = cmaes.Gen.to_numpy()
m = m+1

fig = figure(figsize=(6,5))
scatter(cmaes.RadX.to_list(),cmaes.RadY.to_list(),c=m)
cb = colorbar()
cb.set_ticklabels(m)
xlim(-1.1,1.1)
ylim(-1.1,1.1)
savefig(savepath+DF_name+'.svg',format='svg')